In [149]:
# import libraries
import pymongo
import pandas as pd
from datetime import datetime
from dateutil.parser import parse

In [150]:
# Read sales data-set into dataframe
sales_dataset_df = pd.read_csv("Resources/sales data-set.csv")

# Display dataframe
sales_dataset_df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,05/02/2010,24924.50,False
1,1,1,12/02/2010,46039.49,True
2,1,1,19/02/2010,41595.55,False
3,1,1,26/02/2010,19403.54,False
4,1,1,05/03/2010,21827.90,False


In [151]:
# Check data types of sales_dataset_df
sales_dataset_df.dtypes

Store             int64
Dept              int64
Date             object
Weekly_Sales    float64
IsHoliday          bool
dtype: object

In [152]:
# Choose the columns needed, drop IsHoliday column
filtered_sales_dataset_df = sales_dataset_df[['Store', 'Dept', 'Date', 'Weekly_Sales']].copy()

# Convert Date column to datetime 
filtered_sales_dataset_df['Date'] = pd.to_datetime(filtered_sales_dataset_df['Date'])

# Display dataframe
filtered_sales_dataset_df.head()

,Store,Dept,Date,Weekly_Sales
0,1,1,2010-05-02,24924.50
1,1,1,2010-12-02,46039.49
2,1,1,2010-02-19,41595.55
3,1,1,2010-02-26,19403.54
4,1,1,2010-05-03,21827.90


In [153]:
# Choose only Store, Dept and Date columns to calculate total dept within a store and given date
filtered_sales_dept_df = filtered_sales_dataset_df[['Store', 'Dept', 'Date']]

# Calculate no. of Dept
filtered_sales_dept_df = filtered_sales_dept_df.groupby(['Store', 'Date']).count().reset_index()

# Rename Dept column to Total_Dept since its show the total Dept 
filtered_sales_dept_df.rename(columns = {'Dept':'Total_Dept'}, inplace = True)

# Display dataframe
filtered_sales_dept_df.head()

,Store,Date,Total_Dept
0,1,2010-01-10,73
1,1,2010-02-04,72
2,1,2010-02-07,73
3,1,2010-02-19,73
4,1,2010-02-26,72


In [154]:
# Calculate the total weekly sales by Store and Date
filtered_sales_dataset_df_grpby = filtered_sales_dataset_df.groupby(['Store', 'Date']).sum().reset_index()

# Display dataframe
filtered_sales_dataset_df_grpby.head()

,Store,Date,Dept,Weekly_Sales
0,1,2010-01-10,3233,1453329.50
1,1,2010-02-04,3137,1594968.28
2,1,2010-02-07,3314,1492418.14
3,1,2010-02-19,3184,1611968.17
4,1,2010-02-26,3137,1409727.59


In [155]:
# Read Features data set into dataframe
features_dataset_df = pd.read_csv("Resources/Features data set.csv")

# Display dataframe
features_dataset_df.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,12/02/2010,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,19/02/2010,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,26/02/2010,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,05/03/2010,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [156]:
# Check data types of features_dataset_df
features_dataset_df.dtypes

Store             int64
Date             object
Temperature     float64
Fuel_Price      float64
MarkDown1       float64
MarkDown2       float64
MarkDown3       float64
MarkDown4       float64
MarkDown5       float64
CPI             float64
Unemployment    float64
IsHoliday          bool
dtype: object

In [157]:
# Choose only Store, Date and MarkdownX columns
filtered_features_dataset_df = features_dataset_df[['Store', 'Date', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']].copy()

# Convert Date column to datetime as its in string
filtered_features_dataset_df['Date'] = pd.to_datetime(filtered_features_dataset_df['Date'])

# Display dataframe
filtered_features_dataset_df.head()

,Store,Date,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5
0,1,2010-05-02,NaN,NaN,NaN,NaN,NaN
1,1,2010-12-02,NaN,NaN,NaN,NaN,NaN
2,1,2010-02-19,NaN,NaN,NaN,NaN,NaN
3,1,2010-02-26,NaN,NaN,NaN,NaN,NaN
4,1,2010-05-03,NaN,NaN,NaN,NaN,NaN


In [158]:
# Fill NaN with 0
filtered_features_dataset_df = filtered_features_dataset_df.fillna(0.00)

# Display dataframe
filtered_features_dataset_df.head()

,Store,Date,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5
0,1,2010-05-02,0.0,0.0,0.0,0.0,0.0
1,1,2010-12-02,0.0,0.0,0.0,0.0,0.0
2,1,2010-02-19,0.0,0.0,0.0,0.0,0.0
3,1,2010-02-26,0.0,0.0,0.0,0.0,0.0
4,1,2010-05-03,0.0,0.0,0.0,0.0,0.0


In [159]:
# Merge two dataframes using an outer join and drop extra rows as 2 dataframes aren't the same size
merge_weeklysales = filtered_sales_dataset_df_grpby.merge(filtered_features_dataset_df, on=["Store", "Date"], how="outer").dropna()

# Merge dataframe to Total Dept dataframe 
merge_weeklysales = merge_weeklysales.merge(filtered_sales_dept_df, on=["Store", "Date"], how="outer")

# Drop Dept column after merge
merge_weeklysales = merge_weeklysales[['Store', 'Date', 'Weekly_Sales', 'MarkDown1',
                                     'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'Total_Dept']]
# Display dataframe
merge_weeklysales.head()

,Store,Date,Weekly_Sales,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,Total_Dept
0,1,2010-01-10,1453329.50,0.0,0.0,0.0,0.0,0.0,73
1,1,2010-02-04,1594968.28,0.0,0.0,0.0,0.0,0.0,72
2,1,2010-02-07,1492418.14,0.0,0.0,0.0,0.0,0.0,73
3,1,2010-02-19,1611968.17,0.0,0.0,0.0,0.0,0.0,73
4,1,2010-02-26,1409727.59,0.0,0.0,0.0,0.0,0.0,72


In [160]:
# Verify dataframe size is correct
merge_weeklysales.count()

Store           6435
Date            6435
Weekly_Sales    6435
MarkDown1       6435
MarkDown2       6435
MarkDown3       6435
MarkDown4       6435
MarkDown5       6435
Total_Dept      6435
dtype: int64

In [161]:
# Rename MarkDownX columns to Mark_Down_X
merge_weeklysales.rename(columns = {'MarkDown1':'Mark_Down_1', 'MarkDown2':'Mark_Down_2', 'MarkDown3':'Mark_Down_3', 
                                    'MarkDown4':'Mark_Down_4', 'MarkDown5':'Mark_Down_5'}, inplace = True)

# Display dataframe
merge_weeklysales.head()

,Store,Date,Weekly_Sales,Mark_Down_1,Mark_Down_2,Mark_Down_3,Mark_Down_4,Mark_Down_5,Total_Dept
0,1,2010-01-10,1453329.50,0.0,0.0,0.0,0.0,0.0,73
1,1,2010-02-04,1594968.28,0.0,0.0,0.0,0.0,0.0,72
2,1,2010-02-07,1492418.14,0.0,0.0,0.0,0.0,0.0,73
3,1,2010-02-19,1611968.17,0.0,0.0,0.0,0.0,0.0,73
4,1,2010-02-26,1409727.59,0.0,0.0,0.0,0.0,0.0,72


In [162]:
# storing the cleaned data to MongoDB

In [163]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)


In [164]:
# Define the 'weeklysales DB' database in Mongo
db = client.weeklysales_db
#Declare the collection
collection = db.weeklysales_db

In [165]:
# Check client connection
weeklysales = db.weeklysales.find()

In [166]:
# Check weeklysales query result
weeklysales

In [168]:
# Convert dataframe to dictionary record type to upload to MongoDB
results = merge_weeklysales.to_dict('records')

In [169]:
# Display dictionary records
results

[{'Store': 1,
  'Date': Timestamp('2010-01-10 00:00:00'),
  'Weekly_Sales': 1453329.4999999998,
  'Mark_Down_1': 0.0,
  'Mark_Down_2': 0.0,
  'Mark_Down_3': 0.0,
  'Mark_Down_4': 0.0,
  'Mark_Down_5': 0.0,
  'Total_Dept': 73},
 {'Store': 1,
  'Date': Timestamp('2010-02-04 00:00:00'),
  'Weekly_Sales': 1594968.2799999998,
  'Mark_Down_1': 0.0,
  'Mark_Down_2': 0.0,
  'Mark_Down_3': 0.0,
  'Mark_Down_4': 0.0,
  'Mark_Down_5': 0.0,
  'Total_Dept': 72},
 {'Store': 1,
  'Date': Timestamp('2010-02-07 00:00:00'),
  'Weekly_Sales': 1492418.1399999997,
  'Mark_Down_1': 0.0,
  'Mark_Down_2': 0.0,
  'Mark_Down_3': 0.0,
  'Mark_Down_4': 0.0,
  'Mark_Down_5': 0.0,
  'Total_Dept': 73},
 {'Store': 1,
  'Date': Timestamp('2010-02-19 00:00:00'),
  'Weekly_Sales': 1611968.17,
  'Mark_Down_1': 0.0,
  'Mark_Down_2': 0.0,
  'Mark_Down_3': 0.0,
  'Mark_Down_4': 0.0,
  'Mark_Down_5': 0.0,
  'Total_Dept': 73},
 {'Store': 1,
  'Date': Timestamp('2010-02-26 00:00:00'),
  'Weekly_Sales': 1409727.5900000003,
  'M

In [170]:
collection.insert_many(results)
print(results)


[{'Store': 1, 'Date': Timestamp('2010-01-10 00:00:00'), 'Weekly_Sales': 1453329.4999999998, 'Mark_Down_1': 0.0, 'Mark_Down_2': 0.0, 'Mark_Down_3': 0.0, 'Mark_Down_4': 0.0, 'Mark_Down_5': 0.0, 'Total_Dept': 73, '_id': ObjectId('5ecdbcfc25247c8d34c12eb4')}, {'Store': 1, 'Date': Timestamp('2010-02-04 00:00:00'), 'Weekly_Sales': 1594968.2799999998, 'Mark_Down_1': 0.0, 'Mark_Down_2': 0.0, 'Mark_Down_3': 0.0, 'Mark_Down_4': 0.0, 'Mark_Down_5': 0.0, 'Total_Dept': 72, '_id': ObjectId('5ecdbcfc25247c8d34c12eb5')}, {'Store': 1, 'Date': Timestamp('2010-02-07 00:00:00'), 'Weekly_Sales': 1492418.1399999997, 'Mark_Down_1': 0.0, 'Mark_Down_2': 0.0, 'Mark_Down_3': 0.0, 'Mark_Down_4': 0.0, 'Mark_Down_5': 0.0, 'Total_Dept': 73, '_id': ObjectId('5ecdbcfc25247c8d34c12eb6')}, {'Store': 1, 'Date': Timestamp('2010-02-19 00:00:00'), 'Weekly_Sales': 1611968.17, 'Mark_Down_1': 0.0, 'Mark_Down_2': 0.0, 'Mark_Down_3': 0.0, 'Mark_Down_4': 0.0, 'Mark_Down_5': 0.0, 'Total_Dept': 73, '_id': ObjectId('5ecdbcfc25247c8d

In [171]:
# Query all weekly sales
weeklysales = db.weeklysales.find()

# Iterate through each student in the collection
for x in weeklysales:
    print(x)